In [59]:
import skolenhetsregister as sker
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
from ipywidgets import interact, interactive
from pandas_profiling import ProfileReport

%matplotlib inline

In [60]:
with open("national_test_results_grundskola.json", "r") as f:
    data = json.load(f)

In [61]:
# df = pd.read_json(data)
df = pd.DataFrame.from_dict(data)
df.head()

,kommunkod,school_form,year,grade,national_test_results
0,1440,11,2019,6,"{'Svenska': {'boys_average': 11.7, 'girls_aver..."
1,1489,11,2019,6,"{'Svenska': {'boys_average': 12.7, 'girls_aver..."
2,0764,11,2019,6,"{'Svenska': {'boys_average': 11.7, 'girls_aver..."
3,0604,11,2019,6,"{'Svenska': {'boys_average': 11.4, 'girls_aver..."
4,1984,11,2019,6,"{'Svenska': {'boys_average': 11.1, 'girls_aver..."


In [62]:
# unpack national_test_results
json_struct = json.loads(df.to_json(orient="records"))    
df = pd.json_normalize(json_struct)
df.head()

,kommunkod,school_form,year,grade,national_test_results.Svenska.boys_average,national_test_results.Svenska.girls_average,national_test_results.Svenska.total_average,national_test_results.Svenska som andraspråk.boys_average,national_test_results.Svenska som andraspråk.girls_average,national_test_results.Svenska som andraspråk.total_average,...,national_test_results.Historia.total_average,national_test_results.Kemi.boys_average,national_test_results.Kemi.girls_average,national_test_results.Kemi.total_average,national_test_results.Religionskunskap.boys_average,national_test_results.Religionskunskap.girls_average,national_test_results.Religionskunskap.total_average,national_test_results.Samhällskunskap.boys_average,national_test_results.Samhällskunskap.girls_average,national_test_results.Samhällskunskap.total_average
0,1440,11,2019,6,11.7,14.9,13.4,8.6,11.9,10.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1489,11,2019,6,12.7,14.2,13.5,6.8,9.7,8.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0764,11,2019,6,11.7,14.1,12.9,7.0,11.3,8.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0604,11,2019,6,11.4,13.0,12.3,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1984,11,2019,6,11.1,14.7,12.7,5.6,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
# profile = ProfileReport(df_flat, explorative=True)
# profile

In [84]:
# Graphs
def plot_grades_over_time(kommun, grade, subject, gender):
    results = df.rename(columns={col: ".".join( col.split(".")[1:] ) for col in df.columns if len(col.split(".")) > 2})
    
    # todo: add level logic
    
    global kommun_codes
    if kommun != "All":
        code = [x["Kommunkod"] for x in kommun_codes if x["Namn"] == kommun][0]
        results = results.loc[ df["kommunkod"] == code ]
    
    # Numeric data
    results = results.groupby(["year", "grade"]).mean().reset_index()
    
    if grade != "All":
        results = results.loc[ results['grade'] == grade ]
    
    cols = results.columns
    if subject != "All":
        cols = [col for col in cols if subject.lower() in col.lower()]
        
    if gender != "All":
        cols = [col for col in cols if gender.lower() in col.lower()]
    
    grade_title = grade
    if grade == "All":
        grade_title = "6 and 9"
    ax = results.plot.line(x='year', y=cols, ylim=(10,20), ylabel="score", rot=0, title=f"National test results over time in grade {grade_title}")
    ax.axhline(y=17.5, xmin=-1, xmax=1, color='r', linestyle='--', lw=1, label='B')
    ax.axhline(y=15, xmin=-1, xmax=1, color='r', linestyle='--', lw=1, label='C')
    ax.axhline(y=12.5, xmin=-1, xmax=1, color='r', linestyle='--', lw=1, label='D')
    plt.legend()

#levels = ['Grundskolan', 'Gymnasium']
with open("kommunkoder.json", "r") as file:
    kommun_codes = json.load(file)
kommun_names = ["All"] + [x["Namn"] for x in kommun_codes]
grades = ["All", "6", "9"]
subjects = ["Svenska.", "Svenska\u00a0som\u00a0andraspr\u00e5k.", "Matematik.", "Engelska.", "All"]
genders = ["All", "Girls", "Boys"]

interact(plot_grades_over_time, kommun=kommun_names, grade=grades, subject=subjects, gender=genders)

interactive(children=(Dropdown(description='kommun', options=('All', 'Ale', 'Alingsås', 'Alvesta', 'Aneby', 'A…

<function __main__.plot_grades_over_time(kommun, grade, subject, gender)>